### Initial modeling

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Impute Age with MICE

In [3]:
train_mat = train.select_dtypes(include=['number']).as_matrix() #selecting quant vars from train and test and converting to mat
test_mat = test.select_dtypes(include=['number']).as_matrix()

In [4]:
from fancyimpute import MICE

mice = MICE(n_imputations=100,impute_type='col')

train_imputed = mice.complete(train_mat)
test_imputed = mice.complete(test_mat)

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Gwendolyn Fernandez\AppData\Local\Continuum\anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


[MICE] Completing matrix with shape (891, 7)
[MICE] Starting imputation round 1/110, elapsed time 0.001
[MICE] Starting imputation round 2/110, elapsed time 1.968
[MICE] Starting imputation round 3/110, elapsed time 1.969
[MICE] Starting imputation round 4/110, elapsed time 1.970
[MICE] Starting imputation round 5/110, elapsed time 1.971
[MICE] Starting imputation round 6/110, elapsed time 1.971
[MICE] Starting imputation round 7/110, elapsed time 1.972
[MICE] Starting imputation round 8/110, elapsed time 1.972
[MICE] Starting imputation round 9/110, elapsed time 1.973
[MICE] Starting imputation round 10/110, elapsed time 1.974
[MICE] Starting imputation round 11/110, elapsed time 1.974
[MICE] Starting imputation round 12/110, elapsed time 1.975
[MICE] Starting imputation round 13/110, elapsed time 1.975
[MICE] Starting imputation round 14/110, elapsed time 1.976
[MICE] Starting imputation round 15/110, elapsed time 1.976
[MICE] Starting imputation round 16/110, elapsed time 1.977
[MIC

In [5]:
train_cols = train[['PassengerId','Survived','Pclass','Age','SibSp','Parch','Fare']].columns
test_cols = test[['PassengerId','Pclass','Age','SibSp','Parch','Fare']].columns

train_imputed = pd.DataFrame(train_imputed, columns=train_cols) #converting mice output back to pd dataframe
test_imputed = pd.DataFrame(test_imputed, columns=test_cols)

In [6]:
train_imputed['Age'].isnull().value_counts() #all missing values successfully imputed!

False    891
Name: Age, dtype: int64

In [7]:
print('Pre-Imputation Age Mean:',train['Age'].mean())
print('Post-Imputation Age Mean:',train_imputed['Age'].mean())
print('Pre-Imputation SD:',train['Age'].std())
print('Post-Imputation SD:',train_imputed['Age'].std())

Pre-Imputation Age Mean: 29.69911764705882
Post-Imputation Age Mean: 29.62748313689189
Pre-Imputation SD: 14.526497332334044
Post-Imputation SD: 13.023623383981858


Add categoricals back to dataframes with imputed age - * except for cabin *

In [8]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
train_imputed['Name'] = train['Name']
train_imputed['Sex'] = train['Sex']
train_imputed['Ticket'] = train['Ticket']
train_imputed['Embarked'] = train['Embarked']

In [10]:
train_imputed.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Embarked
0,1.0,0.0,3.0,22.0,1.0,0.0,7.2500,"Braund, Mr. Owen Harris",male,A/5 21171,S
1,2.0,1.0,1.0,38.0,1.0,0.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C
2,3.0,1.0,3.0,26.0,0.0,0.0,7.9250,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,S
3,4.0,1.0,1.0,35.0,1.0,0.0,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,S
4,5.0,0.0,3.0,35.0,0.0,0.0,8.0500,"Allen, Mr. William Henry",male,373450,S


In [11]:
test_imputed['Name'] = test['Name']
test_imputed['Sex'] = test['Sex']
test_imputed['Ticket'] = test['Ticket']
test_imputed['Embarked'] = test['Embarked']

In [12]:
test_imputed.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Embarked
0,892.0,3.0,34.5,0.0,0.0,7.8292,"Kelly, Mr. James",male,330911,Q
1,893.0,3.0,47.0,1.0,0.0,7.0000,"Wilkes, Mrs. James (Ellen Needs)",female,363272,S
2,894.0,2.0,62.0,0.0,0.0,9.6875,"Myles, Mr. Thomas Francis",male,240276,Q
3,895.0,3.0,27.0,0.0,0.0,8.6625,"Wirz, Mr. Albert",male,315154,S
4,896.0,3.0,22.0,1.0,1.0,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,3101298,S


Label encoding categorical variables: Name, Sex, Ticket, Embarked

In [13]:
train_x = train_imputed.copy()
train_x = train_x.drop(['Survived'], axis=1)
train_x = train_x.drop(['PassengerId'], axis=1)

In [14]:
test = test_imputed.copy()
test = test.drop(['PassengerId'],axis=1)

In [15]:
print(train_x.shape)
print(test.shape)

(891, 9)
(418, 9)


In [16]:
ntrain = train_x.shape[0]
ntest = test.shape[0]

In [17]:
all_data = pd.concat([train_x,test]).reset_index(drop=True)
all_data.shape

(1309, 9)

In [18]:
cols = ('Name','Sex','Ticket','Embarked')

In [19]:
from sklearn.preprocessing import LabelEncoder

label_df = all_data.copy()

for c in cols:
    if label_df[c].dtype == 'object':
        le = LabelEncoder()
        label_df[c] = le.fit_transform(label_df[c].astype(str))

In [20]:
label_df.head()

,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Embarked
0,3.0,22.0,1.0,0.0,7.2500,155,1,720,2
1,1.0,38.0,1.0,0.0,71.2833,286,0,816,0
2,3.0,26.0,0.0,0.0,7.9250,523,0,914,2
3,1.0,35.0,1.0,0.0,53.1000,422,0,65,2
4,3.0,35.0,0.0,0.0,8.0500,22,1,649,2


In [21]:
label_df.dtypes

Pclass      float64
Age         float64
SibSp       float64
Parch       float64
Fare        float64
Name          int64
Sex           int64
Ticket        int64
Embarked      int64
dtype: object

In [22]:
all_data = label_df

In [23]:
trainX = all_data[:ntrain]
test_x = all_data[ntrain:]

In [24]:
print(trainX.shape)
print(test_x.shape)

(891, 9)
(418, 9)


In [25]:
train_y = train['Survived']

### Random Forest Classifier

Default settings for model and no class weights

In [26]:
from sklearn.ensemble import RandomForestClassifier

basic_rf = RandomForestClassifier(n_jobs=-1) #training first model using default settings and NO class weights
basic_rf.fit(trainX,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [40]:
predictions = basic_rf.predict(trainX) #predicting on training set

In [41]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(train_y, predictions)

In [42]:
conf_mat

array([[549,   0],
       [ 11, 331]], dtype=int64)

In [44]:
from sklearn.metrics import roc_auc_score

auROC = roc_auc_score(train_y, predictions)
auROC

0.98391812865497075

In [46]:
from sklearn.metrics import average_precision_score

precision = average_precision_score(train_y, predictions)
precision

0.98018193632228723

Achieving high accuracy and precision with very basic model.. let's try a grid search to tune hyperparameters (again with NO class weights)

In [47]:
from sklearn.model_selection import GridSearchCV

paramgrid = [{'n_estimators':[200,500,600],'max_features':[2,3,4],'max_depth':[2,3,4,5]}]
grid_search = GridSearchCV(basic_rf, paramgrid, cv=5)
grid_search.fit(trainX,train_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [200, 500, 600], 'max_features': [2, 3, 4], 'max_depth': [2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [48]:
grid_search.best_params_

{'max_depth': 5, 'max_features': 4, 'n_estimators': 200}

In [51]:
best_rf = RandomForestClassifier(n_jobs=-1, max_depth=5, max_features=4, n_estimators=200)
best_rf.fit(trainX,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=4, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [52]:
predictions = best_rf.predict(trainX)

In [54]:
conf_mat = confusion_matrix(train_y, predictions)
conf_mat

array([[528,  21],
       [ 87, 255]], dtype=int64)

In [56]:
auROC = roc_auc_score(train_y, predictions)
auROC

0.85368133448375028

In [57]:
precision = average_precision_score(train_y, predictions)
precision

0.7865256300610991

Best model achieved through CV now performing worse than basic model in terms of accuracy and precision... let's try adding class weights